In [1]:
import pandas as pd
import numpy as np
import folium
import time
import random
from openrouteservice import client
from shapely.geometry import LineString, mapping, Point
from shapely.ops import cascaded_union, unary_union

In [2]:
crime = pd.read_csv("Data/NYPD_Complaint_Folium.csv")
crime

,id,start_date_of_event,start_time_of_event,description,boro,lat,lon,victim_age,victim_race,victim_sex,month,day,year,lat_rnd,lon_rnd
0,134037758,12/26/2019,20:00:00,Petit Larceny,Queens,40.611570,-73.747365,25-44,White Hispanic,F,12,26,2019,40.612,-73.747
1,855385879,12/26/2019,19:57:00,Offenses Against Public Order,Bronx,40.840234,-73.917684,18-24,Black,F,12,26,2019,40.840,-73.918
2,241602326,12/25/2019,23:00:00,Petit Larceny,Manhattan,40.772875,-73.989542,45-64,Black,M,12,25,2019,40.773,-73.990
3,249276017,12/20/2019,14:00:00,Felony Assault,Bronx,40.822271,-73.911698,<18,Black Hispanic,M,12,20,2019,40.822,-73.912
4,636997863,12/20/2019,22:18:00,Reckless Assault,Bronx,40.863223,-73.890411,18-24,Black,F,12,20,2019,40.863,-73.890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4012094,744350707,12/17/2018,03:00:00,Vehicle And Traffic Laws,Bronx,40.887333,-73.847250,25-44,White Hispanic,F,12,17,2018,40.887,-73.847
4012095,380927470,06/05/2018,23:30:00,Grand Larceny Of Motor Vehicle,Bronx,40.827414,-73.877946,25-44,Black,M,6,5,2018,40.827,-73.878
4012096,502083804,02/10/2018,08:30:00,Petit Larceny,Bronx,40.834528,-73.903836,45-64,Black,F,2,10,2018,40.835,-73.904
4012097,609916130,12/11/2018,07:30:00,Reckless Assault,Manhattan,40.811812,-73.938329,<18,Black,M,12,11,2018,40.812,-73.938


In [3]:
crime_latlon = pd.DataFrame(crime.groupby(["lat_rnd", "lon_rnd"]).size().sort_values(ascending = False).head(100)).reset_index()

In [4]:
crime_latlon.drop(columns = 0, inplace = True)

In [5]:
crime_latlon["lat_end"] = crime_latlon["lat_rnd"] + 0.001

In [6]:
crime_latlon["lon_end"] = crime_latlon["lon_rnd"] + 0.001

In [7]:
crime_latlon

,lat_rnd,lon_rnd,lat_end,lon_end
0,40.869,-73.880,40.870,-73.879
1,40.737,-73.983,40.738,-73.982
2,40.823,-73.870,40.824,-73.869
3,40.576,-73.976,40.577,-73.975
4,40.810,-73.925,40.811,-73.924
...,...,...,...,...
95,40.757,-73.971,40.758,-73.970
96,40.701,-73.942,40.702,-73.941
97,40.668,-73.907,40.669,-73.906
98,40.816,-73.868,40.817,-73.867


In [ ]:
crime_latlon.to_csv("NYPD_Complaint_Latlon.csv", index = False)

In [8]:
crime_start = [list(x) for x in zip(crime_latlon["lon_rnd"], crime_latlon["lat_rnd"])]

In [9]:
crime_end = [list(x) for x in zip(crime_latlon["lon_end"], crime_latlon["lat_end"])]

In [10]:
crime_hotspot = [list(x) for x in list(zip(crime_start, crime_end))]

In [11]:
def style_function(color):
    return lambda feature: dict(color = color, opacity = 0.5, weight = 4)

In [12]:
# Basic parameters
api_key = "5b3ce3597851110001cf624828c58c7587b54329a3344bf85f2df83a"  
ors = client.Client(key = api_key)

In [13]:
base_map = folium.Map(location = [40.776676, -73.971321], tiles = "cartodbpositron", zoom_start = 12)

In [14]:
popup_route = "<h4>{0} route</h4><hr>" \
              "<strong>Duration: </strong>{1:.1f} mins<br>" \
              "<strong>Distance: </strong>{2:.3f} km"

In [15]:
# Coordinates for the start and end points for route requests
coordinates = [[-73.989723, 40.741112], [-73.991667, 40.757500]]

In [17]:
# Request route
direction_params = {"coordinates": coordinates,
                    "profile": "foot-walking",
                    "format_out": "geojson",
                    "preference": "shortest",
                    "geometry": "true"}
regular_route = ors.directions(**direction_params)
# Build popup
distance, duration = regular_route["features"][0]["properties"]["summary"].values()
popup = folium.map.Popup(popup_route.format("Regular", duration / 60, distance / 1000))
folium.GeoJson(regular_route, style_function = style_function("blue")).add_child(popup).add_to(base_map)
folium.Marker(list(reversed(coordinates[0])), popup = "Origin").add_to(base_map)
folium.Marker(list(reversed(coordinates[1])), popup = "Destination").add_to(base_map)
base_map

In [16]:
for crime in crime_start[0:50]:
    folium.Marker(list(reversed(crime)), icon = folium.Icon(color = "red"), popup = "Crime Site").add_to(base_map)
base_map

In [20]:
buffer = []
for point in crime_hotspot[0:50]:
    avoid_params = {"coordinates": point,
                    "profile": "foot-walking",
                    "format_out": "geojson",
                    "preference": "shortest",
                    "geometry": "true"}
    avoid_request = ors.directions(**avoid_params)
    time.sleep(1)
    coords = avoid_request["features"][0]["geometry"]["coordinates"]
    route_buffer = LineString(coords).buffer(0.001)  # Create geometry buffer
    # folium.vector_layers.Polygon([(y, x) for x, y in list(route_buffer.exterior.coords)],
    #                              color = ("#FF0000")).add_to(base_map)
    simp_geom = route_buffer.simplify(0.005)  # Simplify geometry for better handling
    buffer.append(simp_geom)
union_buffer = unary_union(buffer)

In [21]:
# New routing with avoided crime hotspots
avoid_crime = {"coordinates": coordinates,
               "format_out": "geojson",
               "profile": "foot-walking",
               "preference": "shortest",
               "instructions": False,
               "options": {"avoid_polygons": mapping(union_buffer)}}
route_crimefree = ors.directions(**avoid_crime)

# Build popup
distance, duration = route_crimefree["features"][0]["properties"]["summary"].values()
popup = folium.map.Popup(popup_route.format("Crimefree Route", duration / 60, distance / 1000))
folium.GeoJson(route_crimefree, style_function = style_function("green")).add_child(popup).add_to(base_map)
base_map.add_child(folium.map.LayerControl())

In [ ]:
landmarks = {"Flatiron Building": [-73.989723, 40.741112],
             "Grand Central Terminal": [-73.977295, 40.752655],
             "Chrysler Building": [-73.975311, 40.751652],
             "Empire State Building": [-73.985428, 40.748817],
             "Rockefeller Center": [-73.978798, 40.758678],
             "One World Trade Center": [-74.013382, 40.712742],
             "Madison Square Garden": [-73.993324, 40.750298],
             "McGraw Hill Building": [-73.991667, 40.757500],
             "Lincoln Center": [-73.984070, 40.772392],
             "The Metropolitan Museum of Art": [-73.963402, 40.779434],
             "The Guggenheim": [-73.958971, 40.782980],
             "Times Square": [-73.985130, 40.758896],
             "Penn Station": [-73.993584, 40.750580]}  

In [ ]:
[landmarks["Flatiron Building"], landmarks["Penn Station"]]

In [ ]:
user_start = "Empire State Building"

In [ ]:
user_end = "Lincoln Center"

In [ ]:
coordinates = [landmarks[user_start], landmarks[user_end]]

In [ ]:
coordinates

In [ ]:
landmark_list = list(landmarks.keys())
landmark_list

In [ ]:
landmark_list